In [1]:
import sys
sys.path.append('../')
from FlowVN import *
sys.path.append('../../')
from CMRx4DFlowMaskGeneration import *
from Utils import *
from einops import rearrange
import matplotlib.pyplot as plt


In [ ]:
! cd ../FlowVN
! python main.py --mode train --loss supervised --network FlowVN --features_in 1 --D_size 15 --T_size 5 --num_act_weights 71 --features_out 8 --kernel_size 5 --num_stages 10 --epoch 50 --lr 0.0001 --batch_size 1

In [ ]:
# Read one subject/case from a directory containing the corresponding .mat files
data_dir = '/mnt/nas/nas3/openData/rawdata/4dFlow/ChallengeData/DemoData'

# `load_mat` takes the .mat file path and variable name (`key`) as input.
# It returns a lazy HDF5-like handle (h5py-style): data is not loaded into memory
# until you index it.
kdata_full = load_mat(data_dir + '/kdata_full.mat', key='kdata_full')
coilmap    = load_mat(data_dir + '/coilmap.mat',    key='coilmap')
segmask    = load_mat(data_dir + '/segmask.mat',    key='segmask')
params     = read_params_csv(data_dir + '/params.csv')

# Use [()] to materialize the entire dataset into RAM as a NumPy array.
kdata_full = kdata_full[()]
coilmap    = coilmap[()]
segmask    = segmask[()]

Nv, Nt, Nc, SPE, PE, FE = kdata_full.shape

VENC = params['VENC']
VOrder = params['venc_order']

In [ ]:
img = np.sum(k2i_numpy(kdata_full, ax=[-1, -2, -3]) * np.conj(coilmap), axis=-4)

In [ ]:
kdata_us = kdata_full * usmask
ksp = rearrange(kdata_us, 'nv nt nc spe pe fe -> nv nc nt fe pe spe')
coils = rearrange(coilmap, 'nc spe pe fe-> nc 1 fe pe spe')